# Klang Strudel Playground

This notebook demonstrates playing Strudel patterns with source location tracking.

In [1]:
USE {
    repositories {
        mavenCentral()
        mavenLocal()
    }

    dependencies {
        // Load the klang-notebook module (which includes all Klang modules)
        implementation("/opt/dev/peekandpoke/klang/klang-notebook/build/libs/klang-notebook-jvm-0.1.0.jar")

        // Also load the dependent modules (since we're not using Maven)
        implementation("/opt/dev/peekandpoke/klang/tones/build/libs/tones-jvm-0.1.0.jar")
        implementation("/opt/dev/peekandpoke/klang/audio_fe/build/libs/audio_fe-jvm-0.1.0.jar")
        implementation("/opt/dev/peekandpoke/klang/audio_bridge/build/libs/audio_bridge-jvm-0.1.0.jar")
        implementation("/opt/dev/peekandpoke/klang/klang/build/libs/klang-jvm-0.1.0.jar")
        implementation("/opt/dev/peekandpoke/klang/klangscript/build/libs/klangscript-jvm-0.1.0.jar")
        implementation("/opt/dev/peekandpoke/klang/strudel/build/libs/strudel-jvm-0.1.0.jar")
    }
}

%use coroutines
%use serialization
%use ktor-client

println("✅ Dependencies loaded!")

✅ Dependencies loaded!


In [2]:
// Import necessary classes
import io.peekandpoke.klang.audio_engine.KlangPlayer
import io.peekandpoke.klang.audio_engine.klangPlayer
import io.peekandpoke.klang.audio_fe.create
import io.peekandpoke.klang.audio_fe.samples.SampleCatalogue
import io.peekandpoke.klang.audio_fe.samples.Samples
import io.peekandpoke.klang.script.klangScript
import io.peekandpoke.klang.strudel.lang.strudelLib
import io.peekandpoke.klang.strudel.StrudelPattern
import kotlinx.coroutines.*

val samples = runBlocking { Samples.create(catalogue = SampleCatalogue.default) }

val player = klangPlayer(
    options = KlangPlayer.Options(
        samples = samples,
    )
)

println("✅ Imports successful!")

Loading sample source: Strudel Default Drums
Loaded sample source: Strudel Default Drums ... Found 1 banks, 16 sounds, 0 aliases
Loading sample source: Tidal Drum Machine
Loaded sample source: Tidal Drum Machine ... Found 71 banks, 683 sounds, 67 aliases
Loading sample source: Dough Samples
Loaded sample source: Dough Samples ... Found 1 banks, 9 sounds, 0 aliases
Loading sample source: Vcsl Samples
Loaded sample source: Vcsl Samples ... Found 29 banks, 128 sounds, 0 aliases
Loading sample source: mridangam
Loaded sample source: mridangam ... Found 1 banks, 13 sounds, 0 aliases
Loading sample source: Piano
Loaded sample source: Piano ... Found 1 banks, 1 sounds, 0 aliases
Loading sample source: GM - Felix Roos
Loaded sample source: GM - Felix Roos ... Found 1 banks, 125 sounds, 0 aliases
[KlangPlayer][JVM] using sample rate 48000
✅ Imports successful!


In [3]:
import io.peekandpoke.klang.strudel.lang.fast
import io.peekandpoke.klang.strudel.lang.gain
import io.peekandpoke.klang.strudel.lang.lpf
import io.peekandpoke.klang.strudel.lang.sound

// Test: Execute a simple Strudel pattern
val song = sound("bd hh sd oh").lpf("100 200 300 400 500 600 700 800").fast(2).gain(1.0)
song

io.peekandpoke.klang.strudel.pattern.ControlPattern@757ca812

In [4]:
// Extract the pattern and query events
import io.peekandpoke.klang.script.runtime.NativeObjectValue

val events = song.queryArc(0.0, 1.0)

println("Found ${events.size} events in first cycle:")
events.forEach { event ->
    val loc = event.sourceLocations?.outermost
    println("  - Event at ${loc?.source}:${loc?.startLine}:${loc?.startColumn}-${loc?.endColumn}")
    println("    Data: ${event.data}")
}

events

Found 8 events in first cycle:
  - Event at null:null:null-null
    Data: VoiceData(note=null, freqHz=null, scale=null, gain=1.0, legato=null, bank=null, sound=bd, soundIndex=null, density=null, panSpread=null, freqSpread=null, voices=null, filters=FilterDefs(filters=[LowPass(cutoffHz=100.0, q=null)]), adsr=AdsrEnvelope(attack=null, decay=null, sustain=null, release=null), accelerate=null, vibrato=null, vibratoMod=null, distort=null, coarse=null, crush=null, cutoff=100.0, hcutoff=null, bandf=null, resonance=null, orbit=null, pan=null, delay=null, delayTime=null, delayFeedback=null, room=null, roomSize=null, begin=null, end=null, speed=null, loop=null, cut=null, value=null)
  - Event at null:null:null-null
    Data: VoiceData(note=null, freqHz=null, scale=null, gain=1.0, legato=null, bank=null, sound=hh, soundIndex=null, density=null, panSpread=null, freqSpread=null, voices=null, filters=FilterDefs(filters=[LowPass(cutoffHz=300.0, q=null)]), adsr=AdsrEnvelope(attack=null, decay=null, su

[StrudelPatternEvent(begin=0.0, end=0.125, dur=0.125, data=VoiceData(note=null, freqHz=null, scale=null, gain=1.0, legato=null, bank=null, sound=bd, soundIndex=null, density=null, panSpread=null, freqSpread=null, voices=null, filters=FilterDefs(filters=[LowPass(cutoffHz=100.0, q=null)]), adsr=AdsrEnvelope(attack=null, decay=null, sustain=null, release=null), accelerate=null, vibrato=null, vibratoMod=null, distort=null, coarse=null, crush=null, cutoff=100.0, hcutoff=null, bandf=null, resonance=null, orbit=null, pan=null, delay=null, delayTime=null, delayFeedback=null, room=null, roomSize=null, begin=null, end=null, speed=null, loop=null, cut=null, value=null), sourceLocations=null), StrudelPatternEvent(begin=0.125, end=0.25, dur=0.125, data=VoiceData(note=null, freqHz=null, scale=null, gain=1.0, legato=null, bank=null, sound=hh, soundIndex=null, density=null, panSpread=null, freqSpread=null, voices=null, filters=FilterDefs(filters=[LowPass(cutoffHz=300.0, q=null)]), adsr=AdsrEnvelope(at

In [ ]:
// Play the pattern
import kotlinx.coroutines.*
import io.peekandpoke.klang.strudel.StrudelPlayback

// Create a playback for the pattern
val playback = StrudelPlayback(
    player = player,
    pattern = song,
    options = StrudelPlayback.Options(cyclesPerSecond = 1.0)
)

runBlocking {
    println("🎵 Starting playback...")
    println("Active playbacks: ${player.activePlaybacks.size}")
    
    playback.start()
    
    println("Playing for 4 seconds...")
    delay(4000)
    
    playback.stop()
    println("✅ Stopped")
}

In [ ]:
// Check if audio system is available
import javax.sound.sampled.AudioSystem

val mixers = AudioSystem.getMixerInfo()
println("Available audio devices:")
mixers.forEach { mixer ->
    println("  - ${mixer.name}: ${mixer.description}")
}

val defaultLine = AudioSystem.getSourceDataLine(
    javax.sound.sampled.AudioFormat(48000f, 16, 2, true, false)
)
println("\nDefault audio line: ${defaultLine.lineInfo}")